In [23]:
import pandas as pd

import plotly.graph_objects as go

pd.options.plotting.backend="plotly"

I had started with [Evidence](https://evidence.dev/) but evidence's experience wasn't working out for me. The query strings needed to be changed at every step to even get it working and there were basically no error messages

## 1. Structure Investigation

A grisly approximation of the shape function in SQL, may Lord have mercy

In [1]:


num_cols = _deepnote_execute_sql("""-- Number of columns 
SELECT COUNT(*) FROM `labocr-267706.eda.INFORMATION_SCHEMA.COLUMNS` WHERE table_name="road_safety"
""", 'SQL_D15DB2FC_356F_4858_A44E_CAB7271219CD')
num_cols

,f0_
0,67


In [2]:


num_rows = _deepnote_execute_sql("""-- Number of rows
SELECT COUNT(*) FROM `labocr-267706.eda.road_safety`
""", 'SQL_D15DB2FC_356F_4858_A44E_CAB7271219CD')
num_rows

,f0_
0,363243


In [3]:


col_types = _deepnote_execute_sql("""-- Count how many times each data type is present in the dataset
SELECT data_type,COUNT(*) FROM `labocr-267706.eda.INFORMATION_SCHEMA.COLUMNS` GROUP BY data_type
""", 'SQL_D15DB2FC_356F_4858_A44E_CAB7271219CD')
col_types

,data_type,f0_
0,STRING,4
1,INT64,6
2,FLOAT64,56
3,DATE,1


### 1.1. Structure of non-numerical features

In [4]:


non_numerical_columns = _deepnote_execute_sql("""SELECT
    STRING_AGG(column_name,",")
  FROM
    `labocr-267706.eda.INFORMATION_SCHEMA.COLUMNS`
  WHERE
    data_type IN ("STRING",
      "DATE")
""", 'SQL_D15DB2FC_356F_4858_A44E_CAB7271219CD')
non_numerical_columns

,f0_
0,"Accident_Index,Date,Time,Local_Authority__High..."


In [5]:


target_columns = _deepnote_execute_sql("""-- Okay I am using a trick with I am not sure would work anywhere else, this is jinjasql 
-- and Deepnote has an integration with SQL to execute this sweetly. 
-- You'll need a certain amount of modification to get this working else where properly
SELECT
  {{non_numerical_columns["f0_"][0] | sqlsafe}}
FROM
  `labocr-267706.eda.road_safety`
""", 'SQL_D15DB2FC_356F_4858_A44E_CAB7271219CD')
target_columns

,Accident_Index,Date,Time,Local_Authority__Highway_,LSOA_of_Accident_Location
0,2015440080946,None,None,None,None
1,2015440080946,None,None,None,None
2,2015440083952,None,None,None,None
3,2015440083952,None,None,None,None
4,2015440101052,None,None,None,None
...,...,...,...,...,...
363238,201597NE00308,2015-08-01,00:58,S12000044,None
363239,201597NE00308,2015-08-01,00:58,S12000044,None
363240,201597UC71106,2015-06-17,09:20,S12000008,None
363241,201597UC71106,2015-06-17,09:20,S12000008,None


In [6]:
target_columns.describe()
# Rewriting describe in SQL is just too much work

,Accident_Index,Date,Time,Local_Authority__Highway_,LSOA_of_Accident_Location
count,363243,319866,319822,319866,298758
unique,140056,365,1439,204,25979
top,201543P296025,2015-02-14,17:30,E10000017,E01028497
freq,1332,2144,2972,8457,1456


### 1.2. Structure of numerical features

In [7]:


numerical_columns = _deepnote_execute_sql("""SELECT
    STRING_AGG(column_name,",")
  FROM
    `labocr-267706.eda.INFORMATION_SCHEMA.COLUMNS`
  WHERE
    data_type IN ("INT64",
      "FLOAT64")
""", 'SQL_D15DB2FC_356F_4858_A44E_CAB7271219CD')
numerical_columns

,f0_
0,"Vehicle_Reference_df_res,Vehicle_Type,Towing_a..."


In [13]:


numericalcols_data = _deepnote_execute_sql("""SELECT
  {% for col in numerical_columns["f0_"][0].split(",") %}
    COUNT(DISTINCT {{col|sqlsafe}}),
  {% endfor %}
FROM
  `labocr-267706.eda.road_safety`
""", 'SQL_D15DB2FC_356F_4858_A44E_CAB7271219CD')
numericalcols_data

,f0_,f1_,f2_,f3_,f4_,f5_,f6_,f7_,f8_,f9_,...,f52_,f53_,f54_,f55_,f56_,f57_,f58_,f59_,f60_,f61_
0,37,20,6,18,10,9,6,12,9,12,...,11,3,11,10,3,5,3,21,3,10


In [33]:
numericalcols_data.columns = numerical_columns["f0_"][0].split(",")
numericalcols_data.T.plot.bar(title="Unique values per feature",log_y=True)

In [19]:
numericalcols_data

,f0_,f1_,f2_,f3_,f4_,f5_,f6_,f7_,f8_,f9_,...,f52_,f53_,f54_,f55_,f56_,f57_,f58_,f59_,f60_,f61_
0,37,20,6,18,10,9,6,12,9,12,...,11,3,11,10,3,5,3,21,3,10


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0250b4f5-a1fd-4aaa-8b27-2847b6649f1e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>